In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
import matplotlib.ticker as mtick
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor

# Filenames

In [3]:
# Pull name
filter_ideo = "Y"
name = "s2_mar23_aug23_ideoY"
t1_follower_set = "s2_mar23_ideoY"
t2_follower_set = "s2_june34_ideoY"

# Initial pull of spreader friends and followers
fr1_fn = "_MINIMAL_FRIENDS_03.18.2023__08.28.43__START0_END-1_merged.csv"
fol1_fn = "__MINIMAL_FOLLOWERS_03.18.2023__08.28.43__START0_END-1_merged.csv"
fol2_fn = "merged_t3_follower_pull.csv"

    
# Hydrated files contain user level metainfo

## Spreader hydrated files 
main_hydrated1_fn = "HydratedMain_ND22___03.28.2023__14.25.56__START0_END-1_merged.csv"
main_hydrated2_fn = "HydratedMain_ND22_t1_t3_v2___10.02.2023__22.42.29__START0_END-1_merged .csv"

## Follower hydrated files 
follower_hydrated1_fn = "HydratedFollowers_ND22___03.28.2023__13.01.40__START0_END-1_merged.csv"
follower_hydrated2_fn = "HydratedFollowers_ND22_t1_t3_v2___10.02.2023__14.23.39__START0_END-1_merged.csv"

# Anchor - date relative to calculate tweet recency to (date when t1 hydrated data pulled)
anchor = "Oct 2 2023"

# Get Ideo

In [7]:
import os

print("\n", "*"*10)
print("STARTING PULLING DATA FOR {}".format(name))
print("*"*10)
print("\n"*2)

ideo_fn = f"processed/{t1_follower_set}_fol1_ideo.csv"
print("Ideo fn", ideo_fn)
print("t2 follower set", t2_follower_set)
print(os.path.exists(ideo_fn))
if filter_ideo == "Y":
    join_type = "inner"
elif filter_ideo == "N":
    join_type = "left"
    
if not os.path.exists(ideo_fn):
    print(f"Getting data for {t1_follower_set}")
    !python3 get_twitter_ideos.py -i $fol1_fn -id_col 'followers_id' --o $ideo_fn --j $join_type
else:
    print(f"The file '{ideo_fn}' already exists.")




 **********
STARTING PULLING DATA FOR s2_mar23_aug23_ideoY
**********



processed/s2_mar23_ideoY_fol1_ideo.csv
s2_june34_ideoY
True
The file 'processed/s2_mar23_ideoY_fol1_ideo.csv' already exists.


# Make T1 df: Columns = [main, follower, recip, theta]

1. FollowerIdeology = FollowerIdeology JOIN Followers
2. FollowerIdeology = FollowerIdeology LEFT JOIN Friends (get reciprocal flag)

In [9]:
print("Making T1 df")
fr_1 = pd.read_csv(fr1_fn, dtype={'main':'object', 'friends_id':'object'})
fr_1['recip'] = 1
fol_1 = pd.read_csv(fol1_fn, dtype={'main':'object', 'followers_id':'object'})

fol_1_ideo = pd.read_csv(ideo_fn, dtype={'main':'object', 'followers_id':'object'})[['followers_id', 'theta']]
fol_1_ideo = pd.merge(fol_1, fol_1_ideo, left_on='followers_id', right_on='followers_id')
fol_1_ideo = fol_1_ideo.drop_duplicates(subset=['followers_id', 'main'])
fol_1_ideo = pd.merge(fol_1_ideo, fr_1, left_on=['main', 'followers_id'], right_on=['main', 'friends_id'], how='left')
fol_1_ideo['recip'] = fol_1_ideo['recip'].apply(lambda x: 1 if x==1.0 else 0)
fol_1_ideo = fol_1_ideo[['main', 'followers_id', 'recip', 'theta']]
if fr1_fn == "-1":
    fol_1_ideo['recip'] = np.NaN
print(f"After making T1 df, there are {len(fol_1_ideo['followers_id'].unique())} followers and {len(fol_1_ideo['main'].unique())} spreaders")

Making T1 df
After making T1 df, there are 944972 followers and 5593 spreaders


# Find banned spreaders and followers

In [ ]:
ban_fn = f"{name}_banned.csv"

print("Getting banned users")

####################
# BANNED SPREADERS
####################
banned_spreaders = []
# First we look in follower pulls for anytime I logged an error with the spreaders
for fn in [fol1_fn, fol2_fn]:
    tdf = pd.read_csv(fn, dtype={'main':'object', 'followers_id':'object'})
    tdf['er'] = tdf['followers_id'].apply(lambda x: 1 if x.startswith('-99') else 0)
    tdf_er = tdf.query("er==1")['main'].tolist()
    banned_spreaders.extend(tdf_er)

# Now we will look at anytime it was not possible to pull follower counts in hydrated data
for fn in [main_hydrated1_fn, main_hydrated2_fn]:
    if fn != "-1":
        tdf = pd.read_csv(fn, dtype={'user_id':'object'})
        banned_spreaders.extend(tdf.query("follower_count<0")['user_id'].tolist())
        banned_spreaders.extend(tdf.query("tweet_count<0")['user_id'].tolist())


####################
# BANNED FOLLOWERS
####################
# For followers, too, we will look at anytime it was not possible to pull follower count
banned_followers = []
for fn in [follower_hydrated1_fn, follower_hydrated2_fn]:
    if fn != "-1":
        tdf = pd.read_csv(fn, dtype={'user_id':'object'})
        banned_followers.extend(tdf.query("follower_count<0")['user_id'].tolist())
        banned_followers.extend(tdf.query("tweet_count<0")['user_id'].tolist())

banned_followers, banned_spreaders = set(banned_followers), set(banned_spreaders)
print(f"Before removing banned, there are {len(fol_1_ideo['followers_id'].unique())} followers and {len(fol_1_ideo['main'].unique())} spreaders")
print(f"There are {len(banned_followers)} removed followers and {len(banned_spreaders)} removed spreaders")

unique_spreaders = set(fol_1_ideo['main'].unique())
intersection_spreaders = unique_spreaders.intersection(banned_spreaders)
print(f"Out of {len(banned_spreaders)} banned spreaders, {len(intersection_spreaders)} are present in the fol_1_ideo DataFrame.")

####################
# WRITE TO DF
####################
ban_data = []
for banned_follower in banned_followers:
    ban_data.append({'user_id':banned_follower, 'type':'follower'})
for banned_spreader in banned_spreaders:
    ban_data.append({'user_id':banned_spreader, 'type':'spreader'})
ban_df = pd.DataFrame(ban_data)
ban_df.to_csv("processed/" + ban_fn)

# Remove any banned spreaders and followers if there are any

In [ ]:
fol_1_ideo = fol_1_ideo[~(fol_1_ideo['main'].isin(banned_spreaders) | fol_1_ideo['followers_id'].isin(banned_followers))]
print(f"After removing banned, there are {len(fol_1_ideo['followers_id'].unique())} followers and {len(fol_1_ideo['main'].unique())} spreaders")
fol_1_ideo.to_csv(f"processed/{name}_fol_1_ideo_remove_ban.csv")

# Add activity data

In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import pytz

def calculate_days_since_tweet(date, reference_date):
    days = (reference_date - date).dt.days
    # Replace negative values with 0. This happens if:
    # - pull begins on Jan 1
    # - gets to user on Jan 5nd
    # - user's last tweet was Jan 3
    # We will get -2 
    days = days.clip(0)  
    return days

def process_dataframe(df, anchor=anchor):
    reference_date = datetime.strptime(anchor, '%b %d %Y').replace(tzinfo=pytz.UTC)
    df['last_tweet_date'] = pd.to_datetime(df['last_tweet_date'], errors='coerce', format='%a %b %d %H:%M:%S %z %Y')
    df = df.dropna(subset=['last_tweet_date'])
    df['days_since_tweet'] = calculate_days_since_tweet(df['last_tweet_date'], reference_date)
    return df


# Set up data
#########################
make_percent = lambda num, denom: str(int((num/denom)*100)) + "%"
fol_1_ideo = pd.read_csv(f"processed/{name}_fol_1_ideo_remove_ban.csv",  dtype={'main':'object', 'followers_id':'object'})
followers = list(set(fol_1_ideo['followers_id'].tolist()))
hydf2 = pd.read_csv(follower_hydrated2_fn, dtype={'user_id':'object'})
hydf2 = hydf2.query("user_id in @ followers")
#########################


# Pull tweet days
#########################
tweet_days_fn = f"{name}_follower_tweet_recency.csv"
if not os.path.exists(tweet_days_fn):
    print("Getting tweet days")
    tweet_days_df = process_dataframe(hydf2)
    tweet_days_errors = len(hydf2) - len(tweet_days_df)
    tweet_days_df[['user_id', 'days_since_tweet']].dropna(subset=['days_since_tweet']).to_csv("processed/" + tweet_days_fn)
    print("Pulled tweet days")
    print(f"Couldn't pull tweet days for {tweet_days_errors} out of {len(hydf2)}, {make_percent(tweet_days_errors, len(hydf2))}")
else:
    print("Tweet days already pulled")



# Add back features

In [ ]:
# READ IN FILES 
##############################
# The resulting dataframe from getting ideology and removing any banned people
fol_1_ideo = pd.read_csv(f"processed/{name}_fol_1_ideo_remove_ban.csv",dtype={'main':'object', 'followers_id':'object'})
print(f"After ideology and ban filters, we start with: {len(fol_1_ideo['followers_id'].unique())} followers and {len(fol_1_ideo['main'].unique())} spreaders")

# The activity features for followers
tweet_days = pd.read_csv(f"processed/{name}_follower_tweet_recency.csv", dtype={'user_id':'object'})

# Hydrated followers and spreaders containing stuff like tweet count
hyds1 = pd.read_csv(main_hydrated1_fn, dtype={'user_id':'object'}).query("follower_count!='-9'")
hydf1 = pd.read_csv(follower_hydrated1_fn, dtype={'user_id':'object'}).query("follower_count!='-9'")

hyds2 = pd.read_csv(main_hydrated2_fn, dtype={'user_id':'object'}).query("follower_count!='-9'")
hydf2 = pd.read_csv(follower_hydrated2_fn, dtype={'user_id':'object'}).query("follower_count!='-9'")

##############################


# ADD FEATURES
##############################
for c in ['follower_count', 'following_count', 'tweet_count', 'last_tweet_date', 'user_id']:
    if c in hyds1.columns:
        hyds1[f"spreader_{c}"] = hyds1[c]
        hyds2[f"t2_spreader_{c}"] = hyds2[c]

    if c in hydf1.columns:
        hydf1[f"follower_{c}"] = hydf1[c]
        hydf2[f"t2_follower_{c}"] = hydf2[c]

# Merge spreader info (T1)
fol_1_ideo = pd.merge(fol_1_ideo, hyds1[['user_id', 'spreader_tweet_count', 'spreader_following_count', 'spreader_follower_count']], left_on=['main'], right_on=['user_id'], how='inner')
print(f"Merge with T1 hydrated spreader info: There are {len(fol_1_ideo['followers_id'].unique())} followers and {len(fol_1_ideo['main'].unique())} spreaders")

# Merge spreader info (T2)
fol_1_ideo = pd.merge(fol_1_ideo, hyds2[['t2_spreader_user_id', 't2_spreader_tweet_count', 't2_spreader_following_count', 't2_spreader_follower_count']], left_on=['main'], right_on=['t2_spreader_user_id'], how='inner')
print(f"Merge with T2 hydrated spreader info: There are {len(fol_1_ideo['followers_id'].unique())} followers and {len(fol_1_ideo['main'].unique())} spreaders")

# Merge follower info (T1)
fol_1_ideo = pd.merge(fol_1_ideo, hydf1[['user_id', 'follower_tweet_count', 'follower_following_count', 'follower_follower_count']], left_on=['followers_id'], right_on=['user_id'], how='inner')
print(f"Merge with T1 hydrated follower info: There are {len(fol_1_ideo['followers_id'].unique())} followers and {len(fol_1_ideo['main'].unique())} spreaders")

# Merge follower info (T2)
fol_1_ideo = pd.merge(fol_1_ideo, hydf2[['t2_follower_user_id', 't2_follower_tweet_count', 't2_follower_following_count', 't2_follower_follower_count']], left_on=['followers_id'], right_on=['t2_follower_user_id'], how='inner')
print(f"Merge with T2 hydrated follower info: There are {len(fol_1_ideo['followers_id'].unique())} followers and {len(fol_1_ideo['main'].unique())} spreaders")


# Remove missing again just to be 100% sure
fol_1_ideo = fol_1_ideo.query("spreader_following_count != -9")
fol_1_ideo = fol_1_ideo.query("follower_following_count != -9")
print(f"Remove missing people (be extra sure): There are {len(fol_1_ideo['followers_id'].unique())} followers and {len(fol_1_ideo['main'].unique())} spreaders")

# # Add back tweet recency filter
# fol_1_ideo = pd.merge(fol_1_ideo, tweet_days, left_on=['followers_id'], right_on=['user_id'], how='inner')
# print(f"Merge with tweet recency data: There are {len(fol_1_ideo['followers_id'].unique())} followers and {len(fol_1_ideo['main'].unique())} spreaders")

fol_1_ideo['n_spreader_following'] = fol_1_ideo.groupby(by=['followers_id'])['followers_id'].transform('count') 

# Get change varaibles
for col in ['spreader_tweet_count', 'spreader_following_count', 'spreader_follower_count', 'follower_tweet_count', 'follower_following_count', 'follower_follower_count']:
    fol_1_ideo[f'change_{col}'] = fol_1_ideo[f't2_{col}'] - fol_1_ideo[col]
    fol_1_ideo[f'change_{col}2'] = fol_1_ideo[f'change_{col}'].apply(lambda x: max(x, 0))

    
# add basic features
fol_1_ideo['abs_ideo'] = fol_1_ideo['theta'].apply(lambda x: np.abs(x))
fol_1_ideo['is_liberal'] = fol_1_ideo['theta'].apply(lambda x: 1 if x < 0 else 0)
##############################


# Find who was unfollowed

In [ ]:
followed_2 = pd.read_csv(fol2_fn, dtype={'main':'object', 'followers_id':'object'})
followed_2 = followed_2[~(followed_2['main'].isin(banned_spreaders) | followed_2['followers_id'].isin(banned_followers))]
final_df = pd.merge(fol_1_ideo, followed_2, left_on=['main', 'followers_id'], right_on=['main', 'followers_id'], how='left', indicator=True)
final_df = final_df.drop_duplicates(subset=['main', 'followers_id'])
# If the follower id is only present in fol_1_ideo but not in followed_2, then we know person unfollowed
final_df['unfollowed'] = final_df['_merge'].apply(lambda x: 1 if x == 'left_only' else 0)
final_df['unfollowed_spreaders'] = final_df.groupby(by=['followers_id'])['unfollowed'].transform('sum')
unf = final_df['unfollowed'].mean()
print(f"The unfollow_rate for {name} is {unf}")
final_df.to_csv(f"processed/{name}_final_mod.csv")